In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 5.3 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id="", client_secret="")

In [ ]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## EXTRACT


###Hot hits Vietnam dataset.

In [ ]:
playlist_link = 'https://open.spotify.com/playlist/37i9dQZF1DX0F4i7Q9pshJ'
playlist_URI = playlist_link.split("/")[-1].split('?')[0]

In [ ]:
data = sp.playlist_tracks(playlist_URI)
data.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

### Extract some infomations.

Get song name.

In [ ]:
data['items'][0]['track']['name']

'có hẹn với thanh xuân'

Get album name.

In [ ]:
data['items'][0]['track']['album']['name']

'Hương Mùa Hè'

Release date.

In [ ]:
data['items'][0]['track']['album']['release_date']

'2022-07-24'

Total tracks.

In [ ]:
data['items'][0]['track']['album']['total_tracks']

17

List of artists.

In [ ]:
data['items'][0]['track']['artists'][0]['name']

'SUNI Hạ Linh'

Get link to the albums.

In [ ]:
data['items'][0]['track']['album']['external_urls']['spotify']

'https://open.spotify.com/album/1w74ARpNNB3ybFNhs4qWmV'

##TRANSFORM AND LOAD.

### 1. Album dataframe.

In [ ]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    artists_element = []
    for artist in row['track']['artists']:
        artist_id = artist['id']
        artist_name = artist['name']
        artist_url = artist['external_urls']['spotify']
        artists_element.append({'artist_id':artist_id,'artist_name':artist_name,'artist_url':artist_url})
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,
                        'total_tracks':album_total_tracks,'url':album_url, 'artists': artists_element}
    album_list.append(album_element)

In [ ]:
album_list[0]

{'album_id': '1w74ARpNNB3ybFNhs4qWmV',
 'name': 'Hương Mùa Hè',
 'release_date': '2022-07-24',
 'total_tracks': 17,
 'url': 'https://open.spotify.com/album/1w74ARpNNB3ybFNhs4qWmV',
 'artists': [{'artist_id': '4x1fUORHa2EsxrQ6ZzAoQ0',
   'artist_name': 'SUNI Hạ Linh',
   'artist_url': 'https://open.spotify.com/artist/4x1fUORHa2EsxrQ6ZzAoQ0'},
  {'artist_id': '6OzE2OdvV2tGAxSBsBuZ74',
   'artist_name': 'Hoàng Dũng',
   'artist_url': 'https://open.spotify.com/artist/6OzE2OdvV2tGAxSBsBuZ74'},
  {'artist_id': '6d0dLenjy5CnR5ZMn2agiV',
   'artist_name': 'GREY D',
   'artist_url': 'https://open.spotify.com/artist/6d0dLenjy5CnR5ZMn2agiV'},
  {'artist_id': '5lAfakPZgxFKgiJD6xAF1G',
   'artist_name': 'Orange',
   'artist_url': 'https://open.spotify.com/artist/5lAfakPZgxFKgiJD6xAF1G'},
  {'artist_id': '3diftVOq7aEIebXKkC34oR',
   'artist_name': 'tlinh',
   'artist_url': 'https://open.spotify.com/artist/3diftVOq7aEIebXKkC34oR'}]}

In [ ]:
album_df = pd.DataFrame.from_dict(album_list)
album_df = album_df.drop_duplicates(subset=['album_id'])
album_df.head()

,album_id,name,release_date,total_tracks,url,artists
0,1w74ARpNNB3ybFNhs4qWmV,Hương Mùa Hè,2022-07-24,17,https://open.spotify.com/album/1w74ARpNNB3ybFN...,"[{'artist_id': '4x1fUORHa2EsxrQ6ZzAoQ0', 'arti..."
1,2AZXUAGWs90yNEUC4biubs,Chìm Sâu,2022-02-24,1,https://open.spotify.com/album/2AZXUAGWs90yNEU...,"[{'artist_id': '1zSv9qZANOWB4HRE8sxeTL', 'arti..."
2,56L3SxM76dbmYCD1tOziHB,Trái đất ôm Mặt trời,2023-12-18,1,https://open.spotify.com/album/56L3SxM76dbmYCD...,"[{'artist_id': '3E6LGptA8lBEXPHAQCE3vr', 'arti..."
3,6TnmFHbUYVfe4iuQjPQB2r,Đừng Làm Trái Tim Anh Đau,2024-06-08,1,https://open.spotify.com/album/6TnmFHbUYVfe4iu...,"[{'artist_id': '5dfZ5uSmzR7VQK0udbAVpf', 'arti..."
4,10FLjwfpbxLmW8c25Xyc2N,Die With A Smile,2024-08-16,1,https://open.spotify.com/album/10FLjwfpbxLmW8c...,"[{'artist_id': '1HY2Jd0NmPuamShAr6KMms', 'arti..."


In [ ]:
album_df['release_date'] = pd.to_datetime(album_df['release_date'], format = 'mixed', dayfirst = 'True')
album_df.head()

,album_id,name,release_date,total_tracks,url,artists
0,1w74ARpNNB3ybFNhs4qWmV,Hương Mùa Hè,2022-07-24,17,https://open.spotify.com/album/1w74ARpNNB3ybFN...,"[{'artist_id': '4x1fUORHa2EsxrQ6ZzAoQ0', 'arti..."
1,2AZXUAGWs90yNEUC4biubs,Chìm Sâu,2022-02-24,1,https://open.spotify.com/album/2AZXUAGWs90yNEU...,"[{'artist_id': '1zSv9qZANOWB4HRE8sxeTL', 'arti..."
2,56L3SxM76dbmYCD1tOziHB,Trái đất ôm Mặt trời,2023-12-18,1,https://open.spotify.com/album/56L3SxM76dbmYCD...,"[{'artist_id': '3E6LGptA8lBEXPHAQCE3vr', 'arti..."
3,6TnmFHbUYVfe4iuQjPQB2r,Đừng Làm Trái Tim Anh Đau,2024-06-08,1,https://open.spotify.com/album/6TnmFHbUYVfe4iu...,"[{'artist_id': '5dfZ5uSmzR7VQK0udbAVpf', 'arti..."
4,10FLjwfpbxLmW8c25Xyc2N,Die With A Smile,2024-08-16,1,https://open.spotify.com/album/10FLjwfpbxLmW8c...,"[{'artist_id': '1HY2Jd0NmPuamShAr6KMms', 'arti..."


In [ ]:
album_df['release_date'][17]

Timestamp('2020-01-01 00:00:00')

### 2. Song dataframe.

In [ ]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    song_element = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,
                    'popularity':song_popularity,'song_added':song_added,'album_id':album_id,
                   }
    song_list.append(song_element)

In [ ]:
song_list[0]

{'song_id': '2qWP5ZarXRsdmxzz3L3BtP',
 'song_name': 'có hẹn với thanh xuân',
 'duration_ms': 222000,
 'url': 'https://open.spotify.com/track/2qWP5ZarXRsdmxzz3L3BtP',
 'popularity': 58,
 'song_added': '2024-09-13T04:00:00Z',
 'album_id': '1w74ARpNNB3ybFNhs4qWmV'}

In [ ]:
song_df = pd.DataFrame.from_dict(song_list)
song_df.head()

,song_id,song_name,duration_ms,url,popularity,song_added,album_id
0,2qWP5ZarXRsdmxzz3L3BtP,có hẹn với thanh xuân,222000,https://open.spotify.com/track/2qWP5ZarXRsdmxz...,58,2024-09-13T04:00:00Z,1w74ARpNNB3ybFNhs4qWmV
1,4ih9noxBfg7euMjCPO0qNH,Chìm Sâu,156000,https://open.spotify.com/track/4ih9noxBfg7euMj...,53,2024-09-13T04:00:00Z,2AZXUAGWs90yNEUC4biubs
2,4NTanSVXBkQ2NmDEbr2vLE,Trái đất ôm Mặt trời,194703,https://open.spotify.com/track/4NTanSVXBkQ2NmD...,59,2024-09-13T04:00:00Z,56L3SxM76dbmYCD1tOziHB
3,31VNCmwspR7nVJ6kruUuJt,Đừng Làm Trái Tim Anh Đau,279208,https://open.spotify.com/track/31VNCmwspR7nVJ6...,68,2024-09-13T04:00:00Z,6TnmFHbUYVfe4iuQjPQB2r
4,2plbrEY59IikOBgBGLjaoe,Die With A Smile,251667,https://open.spotify.com/track/2plbrEY59IikOBg...,100,2024-09-13T04:00:00Z,10FLjwfpbxLmW8c25Xyc2N


In [ ]:
song_df['song_added'] =  pd.to_datetime(song_df['song_added'])
song_df.head()

,song_id,song_name,duration_ms,url,popularity,song_added,album_id
0,2qWP5ZarXRsdmxzz3L3BtP,có hẹn với thanh xuân,222000,https://open.spotify.com/track/2qWP5ZarXRsdmxz...,58,2024-09-13 04:00:00+00:00,1w74ARpNNB3ybFNhs4qWmV
1,4ih9noxBfg7euMjCPO0qNH,Chìm Sâu,156000,https://open.spotify.com/track/4ih9noxBfg7euMj...,53,2024-09-13 04:00:00+00:00,2AZXUAGWs90yNEUC4biubs
2,4NTanSVXBkQ2NmDEbr2vLE,Trái đất ôm Mặt trời,194703,https://open.spotify.com/track/4NTanSVXBkQ2NmD...,59,2024-09-13 04:00:00+00:00,56L3SxM76dbmYCD1tOziHB
3,31VNCmwspR7nVJ6kruUuJt,Đừng Làm Trái Tim Anh Đau,279208,https://open.spotify.com/track/31VNCmwspR7nVJ6...,68,2024-09-13 04:00:00+00:00,6TnmFHbUYVfe4iuQjPQB2r
4,2plbrEY59IikOBgBGLjaoe,Die With A Smile,251667,https://open.spotify.com/track/2plbrEY59IikOBg...,100,2024-09-13 04:00:00+00:00,10FLjwfpbxLmW8c25Xyc2N


### 3. Artist dataframe.

In [ ]:
artist_list = []
for row in data['items']:
    for key, value in row.items():
        if key == "track":
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'], 'artist_name':artist['name'], 'external_url': artist['href']}
                artist_list.append(artist_dict)

In [ ]:
artist_list[0]

{'artist_id': '4x1fUORHa2EsxrQ6ZzAoQ0',
 'artist_name': 'SUNI Hạ Linh',
 'external_url': 'https://api.spotify.com/v1/artists/4x1fUORHa2EsxrQ6ZzAoQ0'}

In [ ]:
artist_df = pd.DataFrame.from_dict(artist_list)
artist_df = artist_df.drop_duplicates(subset=['artist_id'])
artist_df.head()

,artist_id,artist_name,external_url
0,4x1fUORHa2EsxrQ6ZzAoQ0,SUNI Hạ Linh,https://api.spotify.com/v1/artists/4x1fUORHa2E...
1,6OzE2OdvV2tGAxSBsBuZ74,Hoàng Dũng,https://api.spotify.com/v1/artists/6OzE2OdvV2t...
2,6d0dLenjy5CnR5ZMn2agiV,GREY D,https://api.spotify.com/v1/artists/6d0dLenjy5C...
3,5lAfakPZgxFKgiJD6xAF1G,Orange,https://api.spotify.com/v1/artists/5lAfakPZgxF...
4,3diftVOq7aEIebXKkC34oR,tlinh,https://api.spotify.com/v1/artists/3diftVOq7aE...
